# **Creating matrixes which is likely, social pressure, network, opinion and purchase**



1.   Function of the likely matrix
2.   Function of the social pressure matrix
3.   Function of the network matrix
4.   Function of the opinion matrix




In [ ]:
N = 0 # agent
M = 0 # items
k = 0 # k value of network



# Function to create a matrix of random likelihood scores
def create_likely(N, M):
    """
    Create a matrix of random likelihood scores.

    Parameters:
    - N (int): Number of agents.
    - M (int): Number of items.

    Returns:
    - DataFrame: DataFrame representing the likelihood scores.
    """
    matrix_random = np.random.rand(N, M)
    rows = ['a' + str(i) for i in range(1, N + 1)]
    columns = ['b' + str(i) for i in range(1, M + 1)]
    df = pd.DataFrame(matrix_random, index=rows, columns=columns)
    return df

# Function to create the social pressure matrix
def create_socialpressure(N, M, items):
    """
    Create the social pressure matrix based on likelihood scores.

    Parameters:
    - N (int): Number of agents.
    - M (int): Number of items.
    - items (list): List of item names.

    Returns:
    - DataFrame: DataFrame representing the social pressure matrix.
    """

  socialpressure = pd.DataFrame()



  for n in range(len(items)  ):
    temp = items.copy()
    for m in range(N ):

      row = likely.loc['a' + str(m + 1)]


      sorted_row = row.sort_values(ascending=False)

      second_largest_value = sorted_row.iloc[n]
      second_largest_index = sorted_row.index[n]


      socialpressure.loc['t=' + str(n), 'a' + str(m + 1)] = second_largest_index
  return socialpressure

# Function to create the network matrix
def create_network(length, k):
    """
    Create a network matrix representing connections between agents.

    Parameters:
    - length (int): Number of agents.
    - k (int): Number of neighbors each agent connects to on each side.

    Returns:
    - DataFrame: DataFrame representing the network matrix.
    """
    matrix = np.zeros((length, length), dtype=int)

    for i in range(length):
        for j in range(1, k + 1):  # Each node has k neighbors on each side
            matrix[i, (i + j) % length] = 1  # Right neighbor
            matrix[i, (i - j) % length] = 1  # Left neighbor
        matrix[i, i] = 0  # Remove self-connection

    # Convert the matrix to a DataFrame
    df = pd.DataFrame(matrix)

    # Rename rows and columns
    df.columns = [f'a{i+1}' for i in range(length)]
    df.index = [f'a{i+1}' for i in range(length)]

    return df

def create_opinion(N, M, network, socialpressure):
    """
    Create the opinion matrix for agents based on network and social pressure.

    Parameters:
    - N (int): Number of agents.
    - M (int): Number of items.
    - network (DataFrame): DataFrame representing the network matrix.
    - socialpressure (DataFrame): DataFrame representing the social pressure matrix.

    Returns:
    - DataFrame: DataFrame representing the opinion matrix.
    """
  opinion = pd.DataFrame()

  for m in range(int((M) + 1)):
    for k in range(N):
      dictionary = {f'b{i}': 0 for i in range(1,M + 1)}
      for n in range(M):
        (opinion_function(1,m,"b" + str(n+1),"a" + str(k+1),network,socialpressure,dictionary))


      opinion.loc['t=' + str(m), 'a' + str(k + 1)] = str(dictionary)
      #dictionary = {f'b{i}': 0 for i in range(1,M + 1)}
  return opinion

# **Functions of the calculations which is social pressure function, likely function and opinion function.**

In [ ]:
def socialpressure_function(t, item, agent, network, socialpressure, dictionary):
    """
    Calculate social pressure for a specific item and agent at time t.

    Parameters:
    - t (int): Time step.
    - item (str): Name of the item.
    - agent (str): Name of the agent.
    - network (DataFrame): DataFrame representing the network matrix.
    - socialpressure (DataFrame): DataFrame representing the social pressure matrix.
    - dictionary (dict): Dictionary to store social pressure values.

    Returns:
    - dict: Updated dictionary with social pressure values.
    """
    liste = []

    for column_name, value in network[str(agent)].items():
        if value == 1:
            liste.append(column_name)

    temp_socialpressure = socialpressure[liste]

    for n in range(t):
        for k in temp_socialpressure.loc['t=' + str(n)]:
            if k == item:
                dictionary[k] += 1 / len(temp_socialpressure.columns)

    return dictionary[item]

def likely_function(agent, item):
    """
    Get the likelihood score of an item for a specific agent.

    Parameters:
    - agent (str): Name of the agent.
    - item (str): Name of the item.

    Returns:
    - float: Likelihood score of the item for the agent.
    """
    return a.loc[str(agent)][str(item)]

def opinion_function(sensitivity, t, item, agent, network, socialpressure, dictionary):
    """
    Calculate the opinion value for a specific item and agent at time t.

    Parameters:
    - sensitivity (float): Sensitivity of social pressure.
    - t (int): Time step.
    - item (str): Name of the item.
    - agent (str): Name of the agent.
    - network (DataFrame): DataFrame representing the network matrix.
    - socialpressure (DataFrame): DataFrame representing the social pressure matrix.
    - dictionary (dict): Dictionary to store social pressure values.

    Returns:
    - float: Opinion value for the item and agent.
    """
    return (sensitivity * (socialpressure_function(t, str(item), agent, network, socialpressure, dictionary))) + ((1 - sensitivity) * likely_function(str(agent), str(item)))


# **It is a sample code if you examine every matrix one by one**
The following piece of code that helps you examine every matrix and to evaluate results.


In [ ]:
N = 5 # number of agent
M = 5 # number of items
k = 2 # k value of network

likely = create_likely(N,M)
items = list(likely.columns)

socialpressure = create_socialpressure(N,M,items,likely)
network = create_network(N,k)
opinion = create_opinion(N,M,network,socialpressure)

print("It is likely matrix   ------------------")
print(likely)
print("--------------------------------")
print("It is social pressure matrix")
print(socialpressure)
print("--------------------------------")
print("It is network matrix")
print(network)
print("--------------------------------")
print("It is opinion matrix")
print(opinion)
print("---------------------------------")


It is likely matrix   ------------------
          b1        b2        b3        b4        b5
a1  0.441513  0.189983  0.613212  0.277521  0.414547
a2  0.222477  0.614398  0.376505  0.974077  0.347642
a3  0.823350  0.465976  0.198486  0.396326  0.477873
a4  0.422527  0.426777  0.009180  0.484808  0.829108
a5  0.847072  0.049167  0.180589  0.142583  0.794281
--------------------------------
It is social pressure matrix
     a1  a2  a3  a4  a5
t=0  b3  b4  b1  b5  b1
t=1  b1  b2  b5  b4  b5
t=2  b5  b3  b2  b2  b3
t=3  b4  b5  b4  b1  b4
t=4  b2  b1  b3  b3  b2
--------------------------------
It is network matrix
    a1  a2  a3  a4  a5
a1   0   1   1   1   1
a2   1   0   1   1   1
a3   1   1   0   1   1
a4   1   1   1   0   1
a5   1   1   1   1   0
--------------------------------
It is opinion matrix
                                                    a1  \
t=0      {'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0}   
t=1  {'b1': 0.5, 'b2': 0, 'b3': 0, 'b4': 0.25, 'b5'...   
t=2  {'b1': 0.5

# **Examples**

In [ ]:
N = 10 # number of agent
M = 10 # number of items
k = 3 # k value of network

def create_simulation(N,M,k):


  likely = create_likely(N,M)
  items = list(likely.columns)

  socialpressure = create_socialpressure(N,M,items)
  #print(a)
  #print(b)
  network = create_network(N,k)


  #socialpressure_function(5,"b5","a5",c,b)
  opinion = create_opinion(10,10,network,socialpressure)
  return opinion

create_simulation(N,M,k)

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10
t=0,"{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ...","{'b1': 0, 'b2': 0, 'b3': 0, 'b4': 0, 'b5': 0, ..."
t=1,"{'b1': 0.3333333333333333, 'b2': 0, 'b3': 0.16...","{'b1': 0.16666666666666666, 'b2': 0.3333333333...","{'b1': 0.16666666666666666, 'b2': 0.5, 'b3': 0...","{'b1': 0.3333333333333333, 'b2': 0.5, 'b3': 0,...","{'b1': 0.3333333333333333, 'b2': 0.16666666666...","{'b1': 0.16666666666666666, 'b2': 0.1666666666...","{'b1': 0, 'b2': 0.3333333333333333, 'b3': 0.16...","{'b1': 0, 'b2': 0.5, 'b3': 0, 'b4': 0, 'b5': 0...","{'b1': 0.16666666666666666, 'b2': 0.3333333333...","{'b1': 0.3333333333333333, 'b2': 0.16666666666..."
t=2,"{'b1': 0.5, 'b2': 0.16666666666666666, 'b3': 0...","{'b1': 0.3333333333333333, 'b2': 0.5, 'b3': 0....","{'b1': 0.3333333333333333, 'b2': 0.5, 'b3': 0....","{'b1': 0.5, 'b2': 0.5, 'b3': 0, 'b4': 0.166666...","{'b1': 0.6666666666666666, 'b2': 0.16666666666...","{'b1': 0.5, 'b2': 0.3333333333333333, 'b3': 0....","{'b1': 0.16666666666666666, 'b2': 0.5, 'b3': 0...","{'b1': 0.16666666666666666, 'b2': 0.6666666666...","{'b1': 0.3333333333333333, 'b2': 0.33333333333...","{'b1': 0.5, 'b2': 0.3333333333333333, 'b3': 0...."
t=3,"{'b1': 0.5, 'b2': 0.3333333333333333, 'b3': 0....","{'b1': 0.3333333333333333, 'b2': 0.66666666666...","{'b1': 0.3333333333333333, 'b2': 0.66666666666...","{'b1': 0.5, 'b2': 0.5, 'b3': 0.166666666666666...","{'b1': 0.6666666666666666, 'b2': 0.33333333333...","{'b1': 0.5, 'b2': 0.5, 'b3': 0.5, 'b4': 0.1666...","{'b1': 0.16666666666666666, 'b2': 0.6666666666...","{'b1': 0.16666666666666666, 'b2': 0.6666666666...","{'b1': 0.3333333333333333, 'b2': 0.33333333333...","{'b1': 0.5, 'b2': 0.3333333333333333, 'b3': 0...."
t=4,"{'b1': 0.6666666666666666, 'b2': 0.5, 'b3': 0....","{'b1': 0.5, 'b2': 0.8333333333333333, 'b3': 0....","{'b1': 0.3333333333333333, 'b2': 0.83333333333...","{'b1': 0.5, 'b2': 0.5, 'b3': 0.333333333333333...","{'b1': 0.6666666666666666, 'b2': 0.33333333333...","{'b1': 0.6666666666666666, 'b2': 0.5, 'b3': 0....","{'b1': 0.3333333333333333, 'b2': 0.83333333333...","{'b1': 0.3333333333333333, 'b2': 0.83333333333...","{'b1': 0.3333333333333333, 'b2': 0.5, 'b3': 0....","{'b1': 0.6666666666666666, 'b2': 0.33333333333..."
t=5,"{'b1': 0.6666666666666666, 'b2': 0.5, 'b3': 0....","{'b1': 0.6666666666666666, 'b2': 0.83333333333...","{'b1': 0.5, 'b2': 0.8333333333333333, 'b3': 0....","{'b1': 0.6666666666666666, 'b2': 0.5, 'b3': 0....","{'b1': 0.6666666666666666, 'b2': 0.33333333333...","{'b1': 0.6666666666666666, 'b2': 0.5, 'b3': 0....","{'b1': 0.3333333333333333, 'b2': 0.83333333333...","{'b1': 0.5, 'b2': 0.8333333333333333, 'b3': 0....","{'b1': 0.5, 'b2': 0.5, 'b3': 0.5, 'b4': 0.8333...","{'b1': 0.8333333333333333, 'b2': 0.33333333333..."
t=6,"{'b1': 0.6666666666666666, 'b2': 0.83333333333...","{'b1': 0.6666666666666666, 'b2': 0.99999999999...","{'b1': 0.5, 'b2': 0.8333333333333333, 'b3': 0....","{'b1': 0.6666666666666666, 'b2': 0.83333333333...","{'b1': 0.6666666666666666, 'b2': 0.83333333333...","{'b1': 0.6666666666666666, 'b2': 0.99999999999...","{'b1': 0.3333333333333333, 'b2': 0.99999999999...","{'b1': 0.5, 'b2': 0.9999999999999999, 'b3': 0....","{'b1': 0.5, 'b2': 0.8333333333333333, 'b3': 0....","{'b1': 0.8333333333333333, 'b2': 0.83333333333..."
t=7,"{'b1': 0.9999999999999999, 'b2': 0.83333333333...","{'b1': 0.8333333333333333, 'b2': 0.99999999999...","{'b1': 0.6666666666666666, 'b2': 0.83333333333...","{'b1': 0.6666666666666666, 'b2': 0.83333333333...","{'b1': 0.8333333333333333, 'b2': 0.83333333333...","{'b1': 0.8333333333333333, 'b2': 0.